### 네이버 뉴스 sub페이지 크롤링

- 한 개의 sub페이지를 크롤링한 후 함수로 재구성
- 02_1번에서 수집된 코드를 이용해서 여러 sub페이지를 크롤링

In [34]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [35]:
# 뉴스 sub 페이지 문서코드 받아오기
url = 'https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=001&aid=0012576698'

headers = {'User-Agent':
           'Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36'}

result = requests.get(url,headers = headers)


In [36]:
result

<Response [200]>

In [37]:
result.content
# 전송된 소스코드를 반환 - row text 형식으로 반환
# 확인방법: 맨 앞에 b 코드가 붙어 있음

result.text
# 전송된 소스코드를 문자열로 반환

'<!DOCTYPE HTML> \r\n<html lang="ko"> \r\n<head>\r\n<meta charset="euc-kr">\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\r\n<meta name="referrer" contents="always">\r\n<meta name="viewport" content="width=1106" />\r\n<title>정부 "내년도 백신도입 협상 마무리 단계…mRNA 계열 5천만회분" : 네이버 뉴스</title>\r\n\n\n\t\n\t\n\t\t\n\t\n\n\n\t\n\t\n\t\n\t\t\n\t\t\n\t\t\n\t\n\n<meta property="me2:post_tag"\t\tcontent="연합뉴스 : 네이버뉴스"/>\n<meta property="me2:category1"\t\tcontent="연합뉴스"/>\n<meta property="me2:category2"\t\tcontent="경제"/>\n<meta property="me2:image"\t\t\tcontent="https://imgnews.pstatic.net/image/001/2021/08/04/PYH2021073007380001300_P4_20210804114113416.jpg"/>\n\n<meta property="og:title"\t\t\tcontent="정부 &quot;내년도 백신도입 협상 마무리 단계…mRNA 계열 5천만회분&quot;"/>\n<meta property="og:type"\t\t\tcontent="article"/>\n<meta property="og:url"\t\t\t\tcontent="https://news.naver.com/main/read.naver?mode=LSD&mid=sec&oid=001&aid=0012576698&sid1=001"/>\n<meta property="og:image"\t\t\tcontent="https://imgnews.pstatic.

In [38]:
bs_obj = BeautifulSoup(result.content, "html.parser")
# 기사 제목 추출
title = bs_obj.find(id='articleTitle')
# title.text

# 기사 작성시간 추출
date_time = bs_obj.find('span',{'class':'t11'})
# date_time

### 개발자 도구의 'copy selector' 사용방법

- 크롤링하고자 하는 컨텐츠에서 검사 -> 해당 소스에서 우클릭 -> 'copy' - 'copy selector'
    - 주의: 붙여넣기하면 선택자를 반환해주는데, 선택자 사용이 안 되는 경우도 왕왕 있음
        - 잘못된 선택자를 생성했을 경우 : 선택자 재구성
        - 동적으로 생성되는 태그인 경우에는 선택자가 만들어지지만 정적 크롤링은 불가능함
            - 네이버 뉴스 서브페이지의 '좋아요 수'와 기사 댓글 수는 동적으로 만들어지기 때문에 크롤링 불가능함!!
            - '셀레니움' 이라는 패키지를 사용해야 함
            
            

-  좋아요 수 크롤링 결과 ㅡ [ ] 로 나타남 : 사용 불가
    - bs_obj.select("#main_content > div.article_header > div.article_info > div > div.article_btns > div.article_btns_left > div > a > span.u_likeit_text._count.num")

In [39]:
# sub 페이지에서 수집된 데이터를 반환하는 함수 작성(서브페이지 URL을 파라미터로 전달받음)

def get_news_sub_info(url) :
    
    # 1. 전달된 URL에 접속 후 응답객체 반환
    headers = {'User-Agent':
           'Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36'}
    result = requests.get(url,headers = headers)
    
    # 2. bs4 객체 생성
    bs_obj = BeautifulSoup(result.content, "html.parser")
    
    # 3. 필요 데이터 추출
    # 기사 제목 추출(태그 객체 ㅡ h3)
    title = bs_obj.find(id='articleTitle')
    # 기사 작성시간 추출(태그 객체 ㅡ span)
    date_time = bs_obj.find('span',{'class':'t11'})
    
    # 4. 수집한 데이터를 dict로 정리(구성)한 후에 반환
    dict_sub = {}
    dict_sub["기사제목"] = title.text # [key] = value 이용해서 딕셔너리에 새로운 원소를 추가함
    dict_sub["기사입력시간"] = date_time.text
    
    return dict_sub

In [30]:
# main : 함수 호출 코드
# 필요한 함수 가져오기 (02_0_네이버 메뉴, 뉴스에서 발췌)

headers = {"User-Agent":
           "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"} 
url = "https://news.naver.com"
res = requests.get(url,headers=headers)
html = res.text
bs_obj = BeautifulSoup(html,"html.parser") # from bs4 import BeautifulSoup 모듈 import 했기 때문에 bs4. 안 써줘도 됨

# 헤드라인 뉴스의 구성 ul, class : hdline_article_list
news_list = bs_obj.find("ul",{"class":"hdline_article_list"})
# news_list

# 모든 li 추출
lis = news_list.findAll("li")
link_list=[]
for li in lis :
    a_tag = li.find("a")
    link_list.append("https://news.naver.com"+a_tag["href"])
    
# link_list

for url in link_list :
    res = get_news_sub_info(url)
    print(res)

{'기사제목': '쿠오모, 여직원 여럿 성희롱에 고발자 보복까지…조사보고서', '기사입력시간': '2021.08.04. 오전 3:50'}
{'기사제목': '박원순 유족 측 진중권 고소…“성추행 주장은 허위사실”', '기사입력시간': '2021.08.04. 오전 11:21'}
{'기사제목': '이준석, 국민의당 향해 "애송이 취급 말라"며 \'미드\' 추천했다', '기사입력시간': '2021.08.04. 오전 8:43'}
{'기사제목': '윤석열, 반려견 사진으로 \'쩍벌\' 셀프디스…"아빠 유전"', '기사입력시간': '2021.08.04. 오전 11:23'}
{'기사제목': '신규확진 1725명, 닷새 만에 1700명대…수도권 1036명', '기사입력시간': '2021.08.04. 오전 9:55'}


In [ ]:
## 연습문제
# get_news_sub_info 함수의 반환값에 기사 내용까지 포함되게 코드를 수정하고
# 반환되는 결과값을 df로 구성하시오. 

In [53]:
def get_news_subinfo_including_content(url) :
    
    # 1. 전달된 URL에 접속 후 응답객체 반환
    headers = {'User-Agent':
              'Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36'}
    result = requests.get(url, headers = headers)
    
    # 2. bs4 객체 생성
    bs_obj = BeautifulSoup(result.content, "html.parser")
    
    # 3. 필요 데이터 추출
    # 기사 제목 추출(태그 객체 : h3)
    title = bs_obj.find(id='articleTitle')
    # 기사 작성시간 추출(태그 객체 : span)
    date_time = bs_obj.find('span',{'class':'t11'})
    # 기사 본문 추출(태그 객체 : newsct_article)
    content = bs_obj.find("div", {"id" : "articleBodyContents"})
    
    # 4. 수집한 데이터를 dict로 정리(구성)한 후에 반환
    dict_sub = {}
    dict_sub["기사제목"] = title.text
    dict_sub["기사입력시간"] = date_time.text
    dict_sub["기사본문"] = content.text
    
    return dict_sub

In [68]:
import pandas as pd
article = get_news_subinfo_including_content('https://news.naver.com/main/ranking/read.naver?mode=LSD&mid=shm&sid1=001&oid=001&aid=0012576793&rankingType=RANKING')
print(type(article))
list=[]
list.append(article)
df = pd.DataFrame(list)
df

<class 'dict'>


,기사제목,기사입력시간,기사본문
0,"-올림픽- '김연경 28점' 한국, 9년 만에 4강행…45년 만의 메달 도전(종합)",2021.08.04. 오후 12:04,\n\n\n\n\n세계 4위 터키에 3-2 승리…빠르고 높은 공격을 견고한 수비로 ...
